In [1]:
import os

In [2]:
%pwd

'/home/nanshibukawa/Documents/mestrado/disease-classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/nanshibukawa/Documents/mestrado/disease-classification'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/nanshibukawa/disease-classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "nanshibukawa"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "3ff52196ea8c6fbefe8162ac963e4fa611ecc27c"

In [6]:
import tensorflow as tf

2025-07-09 20:01:22.350709: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-09 20:01:22.358808: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-09 20:01:22.368240: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-09 20:01:22.371428: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-09 20:01:22.378458: I tensorflow/core/platform/cpu_feature_guar

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

I0000 00:00:1752102083.763346   28529 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1752102083.800661   28529 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1752102083.802885   28529 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1752102083.807016   28529 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int


In [9]:
from cnnClassifier.constants.constants import *

from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
from cnnClassifier.constants.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.keras",
            training_data = "artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri = "https://dagshub.com/nanshibukawa/disease-classification.mlflow",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

/home/nanshibukawa/Documents/mestrado/disease-classification/.venv/lib/python3.11/site-packages/mlflow/utils/requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _create_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1.0 / 255.0,
            validation_split=0.3
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
            )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self._valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._create_valid_generator()
        self.score = self.model.evaluate(self._valid_generator)
        self.save_score()

    def save_score(self):
        scores = {
            "loss": self.score[0],
            "accuracy": self.score[1],
        }
        save_json(
            path = Path("scores.json"),
            data=scores
        )

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1],
            })


            # --- NEW APPROACH FOR LOGGING MODEL ---
            # 1. Define a temporary path to save the model locally before logging
            # local_model_path = Path("local_model_to_log.keras")
            local_model_path = Path("VGG16Model.keras")
            # 2. Save the model directly using Keras's save method
            # This is where you specify the .keras extension explicitly
            # and avoid the deprecated 'save_format' argument.
            self.model.save(local_model_path)
            print(f"Model temporarily saved to: {local_model_path}")

            # 3. Log the saved model file as an artifact
            # You can still give it a name within MLflow artifacts (e.g., "model.keras")
            mlflow.log_artifact(str(local_model_path), artifact_path="model.keras")

            # 4. If you want to REGISTER the model, use mlflow.register_model
            # This points to the artifact you just logged.
            # Make sure you have the full artifact URI.
            # You might need to construct this based on your run_id
            # For simplicity, for now, we'll just log the artifact.
            # If you specifically need model registration, it's a bit more involved
            # to get the correct URI.
            # A common pattern is:
            logged_model_uri = f"runs:/{mlflow.active_run().info.run_id}/model.keras"
            mlflow.register_model(logged_model_uri, "VGG16Model")
            # --- END NEW APPROACH ---

            # Remove the previous mlflow.keras.log_model calls
            # if tracking_url_type_store != "file":
            #     mlflow.keras.log_model(
            #         self.model,
            #         "model.keras",
            #         registered_model_name="VGG16Model",
            #         keras_model_kwargs={"save_format": "keras"} # THIS IS NOW THE PROBLEM
            #     )
            # else:
            #     mlflow.keras.log_model(
            #         self.model,
            #         "model.keras",
            #         keras_model_kwargs={"save_format": "keras"} # THIS IS NOW THE PROBLEM
            #     )

            # Clean up the local temporary file
            if local_model_path.exists():
                os.remove(local_model_path)
                print(f"Cleaned up local model file: {local_model_path}")



            # if tracking_url_type_store != "file":
            #     mlflow.keras.log_model(
            #         self.model,
            #         "model.keras", # This is the artifact path within MLflow
            #         registered_model_name="VGG16Model",
            #         # ADD THIS ARGUMENT:
            #         keras_model_kwargs={"save_format": "keras"}
            #     )
            #     # mlflow.keras.log_model(self.model, "model.keras", registered_model_name="VGG16Model")

            # else:
            #     mlflow.keras.log_model(self.model, "model.keras"
            #         # self.model,
            #         # "model.keras"
            #     )


In [13]:
import logging

logging.basicConfig(level=logging.INFO)

try:

    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:

    raise e

2025-07-09 20:01:24 | INFO     | cnnClassifier.utils.common:read_yaml:35 | YAML file read successfully from config/config.yaml
2025-07-09 20:01:24 | INFO     | cnnClassifier.utils.common:read_yaml:47 | YAML file content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1oT2VFbMnAScIhUcPpP1q0zjcWZtlSSv4/view?usp=sharing', 'source_URL_id': '1oT2VFbMnAScIhUcPpP1q0zjcWZtlSSv4', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.keras'}}
2025-07-09 20:01:24 | INFO     | cnnClassifier.utils.common:read_yaml:35 | YAML file read successfully from params.yam

/home/nanshibukawa/Documents/mestrado/disease-classification/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1752102085.278026   28624 service.cc:146] XLA service 0x75d068016620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752102085.278058   28624 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-07-09 20:01:25.289568: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-09 20:01:25.344229: I external/local_xla/xla/stream_executor/cuda/cu

  3/234 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.4826 - loss: 5.3536

I0000 00:00:1752102091.348198   28624 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


234/234 ━━━━━━━━━━━━━━━━━━━━ 19s 54ms/step - accuracy: 0.4400 - loss: 42.9521
2025-07-09 20:01:43 | INFO     | cnnClassifier.utils.common:save_json:81 | JSON file saved at scores.json
Model temporarily saved to: VGG16Model.keras


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/07/09 20:01:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 7
Created version '7' of model 'VGG16Model'.


Cleaned up local model file: VGG16Model.keras


In [ ]:
 

!nvidia-smi

Wed Jul  9 20:01:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 570.133.07     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    Off |   00000000:01:00.0  On |                  N/A |
| N/A   52C    P0             19W /   80W |    6304MiB /   8188MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

: 